# Churn Prediction Model 

In [15]:
%pip install --upgrade pip
%pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# import libraries 
import pandas as pd 
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [17]:
# load dataset 
df = pd.read_csv("/Users/tonyzhang/Desktop/customerchurn/data/telco_data_clean.csv")

In [18]:
df.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


## 1. Create X and y variables

In [19]:
X = df.drop("Churn", axis=1)
y = df['Churn']

### 1.1 Train test split 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)    

## 2. Decision Tree Classifier 

In [21]:
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=6, min_samples_leaf=8, random_state=0)
dt_model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=0)